Code from Keras/examples/cifra10_cnn.py

Train a simple deep CNN on the CIFAR10 small images dataset.

It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).

--------------------------------------------------------------------------------
Jie: It's slow to train on cpu. 15mims/epoch


In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

Using TensorFlow backend.


In [2]:
batch_size = 32
num_classes = 10
epochs = 100
# epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'
print(save_dir)

/srv/mlstudy/exercises/keras/saved_models


In [3]:
# The data, split between train and test sets:
# Jie 50000 train samples, each is a 3-chanle 32*32 image 
# Jie 10000 test samples, each is a 3-chanle 32*32 image 
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

170500096/170498071 [==============================] - 26s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
# Convert class vectors to binary class matrices.
# Jie: Keras's one-hot-enconding, keras.utils.to_categorical(source, number_of_categories)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)
print(y_test.shape)

(50000, 10)
(10000, 10)


In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    print("Pre process")
    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)
    print("fit")
    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Using real-time data augmentation.
Pre process
fit
Epoch 1/100
1563/1563 [==============================] - 27s 17ms/step - loss: 1.8808 - acc: 0.3081 - val_loss: 1.6073 - val_acc: 0.4291
Epoch 2/100
1563/1563 [==============================] - 22s 14ms/step - loss: 1.5859 - acc: 0.4234 - val_loss: 1.4713 - val_acc: 0.4724
Epoch 3/100
1563/1563 [==============================] - 22s 14ms/step - loss: 1.4625 - acc: 0.4705 - val_loss: 1.2865 - val_acc: 0.5384
Epoch 4/100
1563/1563 [==============================] - 22s 14ms/step - loss: 1.3675 - acc: 0.5102 - val_loss: 1.3158 - val_acc: 0.5226
Epoch 5/100
1563/1563 [==============================] - 22s 14ms/step - loss: 1.2946 - acc: 0.5355 - val_loss: 1.3120 - val_acc: 0.5433
Epoch 6/100
1563/1563 [==============================] - 22s 14ms/step - loss: 1.2337 - acc: 0.5599 - val_loss: 1.1395 - val_acc: 0.5937
Epoch 7/100
1563/1563 [==============================] - 22s 14ms/step - loss: 1.1938 - acc: 0.5747 - val_loss: 1.0691 - val_ac

1563/1563 [==============================] - 22s 14ms/step - loss: 0.7812 - acc: 0.7359 - val_loss: 0.7896 - val_acc: 0.7392
Epoch 61/100
1563/1563 [==============================] - 22s 14ms/step - loss: 0.7783 - acc: 0.7382 - val_loss: 0.6963 - val_acc: 0.7696
Epoch 62/100
1563/1563 [==============================] - 22s 14ms/step - loss: 0.7774 - acc: 0.7401 - val_loss: 0.6764 - val_acc: 0.7712
Epoch 63/100
1563/1563 [==============================] - 22s 14ms/step - loss: 0.7806 - acc: 0.7393 - val_loss: 0.7207 - val_acc: 0.7622
Epoch 64/100
1563/1563 [==============================] - 22s 14ms/step - loss: 0.7769 - acc: 0.7377 - val_loss: 0.6855 - val_acc: 0.7698
Epoch 65/100
1563/1563 [==============================] - 22s 14ms/step - loss: 0.7784 - acc: 0.7432 - val_loss: 0.7286 - val_acc: 0.7547
Epoch 66/100
1563/1563 [==============================] - 22s 14ms/step - loss: 0.7797 - acc: 0.7392 - val_loss: 0.6976 - val_acc: 0.7642
Epoch 67/100
1563/1563 [=======================